# Quicktour

PEFT, önceden eğitilmiş büyük modellere ince ayar yapmak için parametre etkin yöntemler sunar. Geleneksel paradigma, her bir alt görev için bir modelin tüm parametrelerine ince ayar yapmaktır, ancak günümüzde modellerdeki muazzam parametre sayısı nedeniyle bu son derece maliyetli ve pratik değildir. Bunun yerine, daha az sayıda hızlı parametreyi eğitmek veya eğitilebilir parametrelerin sayısını azaltmak için düşük sıralı adaptasyon (LoRA) gibi bir yeniden parametrizasyon yöntemi kullanmak daha verimlidir.

Bu hızlı tur size PEFT'in temel özelliklerini ve tüketici cihazlarında genellikle erişilemeyen büyük modelleri nasıl eğitebileceğinizi veya bu modeller üzerinde nasıl çıkarım yapabileceğinizi gösterecektir.

## Train

Her PEFT yöntemi, bir **PeftModel** oluşturmak için tüm önemli parametreleri saklayan bir `PeftConfig` sınıfı tarafından tanımlanır. Örneğin, LoRA ile eğitmek için bir **LoraConfig** sınıfı yükleyip oluşturun ve aşağıdaki parametreleri belirtin:

- `task_type`: eğitilecek görev (bu durumda diziden diziye dil modelleme) 
- `inference_mode`: modeli çıkarım için kullanıp kullanmayacağınız 
- `r`: düşük sıralı matrislerin boyutu 
- `lora_alpha`: düşük sıralı matrisler için ölçeklendirme faktörü 
- `lora_dropout`: LoRA katmanlarının bırakma olasılığı

In [1]:
!pip install -q peft

In [2]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, 
    r=8, lora_alpha=32, lora_dropout=0.1
)

LoraConfig kurulduktan sonra **get_peft_model()** fonksiyonu ile bir **PeftModel** oluşturun. Transformers kütüphanesinden yükleyebileceğiniz bir temel modeli ve LoRA ile eğitim için bir modelin nasıl yapılandırılacağına ilişkin parametreleri içeren LoraConfig'i alır.

İnce ayar yapmak istediğiniz temel modeli yükleyin.

In [3]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-large")

config.json:   0%|          | 0.00/800 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Bir **PeftModel** oluşturmak için temel modeli ve **peft_config**'i **get_peft_model()** işleviyle sarın. Modelinizdeki eğitilebilir parametrelerin sayısı hakkında bir fikir edinmek için **print_trainable_parameters** metodunu kullanın

In [4]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.1915


bigscience/mt0-large'ın 1.2B parametresinden sadece %0.19'unu eğitiyorsunuz!

Artık modeli Transformers Trainer, Accelerate veya herhangi bir özel PyTorch eğitim döngüsü ile eğitebilirsiniz.

Örneğin, **Trainer** sınıfı ile eğitmek için, bazı eğitim hiper parametreleri ile bir **TrainingArguments** sınıfı oluşturun.

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="Leotrim/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
2024-08-10 18:33:32.605158: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 18:33:32.605293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 18:33:32.725379: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


```
from trasnformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
```

## Save model

Modelinizin eğitimi tamamlandıktan sonra save_pretrained fonksiyonunu kullanarak modelinizi bir dizine kaydedebilirsiniz.

`model.save_pretrained("output_dir")`

Ayrıca push_to_hub fonksiyonunu kullanarak modelinizi Hub'a kaydedebilirsiniz (önce Hugging Face hesabınıza giriş yaptığınızdan emin olun).

```
from huggingface_hub import notebook_login

notebook_login()
model.push_to_hub("your-name/bigscience/mt0-large-lora")
```

Her iki yöntem de yalnızca eğitilen ekstra PEFT ağırlıklarını kaydeder, bu da depolamanın, aktarmanın ve yüklemenin süper verimli olduğu anlamına gelir. Örneğin, LoRA ile eğitilen bu facebook/opt-350m modeli yalnızca iki dosya içerir: adapter_config.json ve adapter_model.safetensors. adapter_model.safetensors dosyası sadece 6,3 MB!

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/PEFT-hub-screenshot.png)

## Inference

**AutoPeftModel** sınıfı ve **from_pretrained** yöntemi ile çıkarım için herhangi bir PEFT eğitimli modeli kolayca yükleyin:

In [6]:
import torch

from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained("ybelkada/opt-350m-lora")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

adapter_config.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [7]:
model = model.to("cuda")
model.eval()
inputs = tokenizer("Preheat the oven to 350 degrees and place the cookie dough", return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)
print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Preheat the oven to 350 degrees and place the cookie dough in the center of the oven.

In a large bowl, combine the flour, baking powder, baking soda, salt, and cinnamon.

In a separate bowl, combine the egg yolks, sugar, and vanilla.




'Preheat the oven to 350 degrees and place the cookie dough in the center of the oven. In a large bowl, combine the flour, baking powder, baking soda, salt, and cinnamon. In a separate bowl, combine the egg yolks, sugar, and vanilla.'

Otomatik konuşma tanıma gibi AutoPeftModelFor sınıfı ile açıkça desteklenmeyen diğer görevler için, görev için bir model yüklemek üzere temel AutoPeftModel sınıfını kullanmaya devam edebilirsiniz.

In [9]:
# from peft import AutoPeftModel

# model = AutoPeftModel.from_pretrained("smangrul/openai-whisper-large-v2-LORA-colab")